# Step 3.1: GitHub Status Checker

In [7]:
import sys
# To import python scripts from other folders
sys.path.append('../')
import json
from ProjectAnalysis import ProjectAnalysis
import os
from os import listdir
from os.path import isfile, join
from collections import Counter
import pandas as pd
import re
from datetime import datetime
results_path="/home/results/GitHub/"
import time
from IPython.display import clear_output

In [8]:
def get_projects_resume():
    projects = []
    for project_name in [p for p in os.listdir(results_path) if os.path.isdir(results_path+p)]:
        pa = ProjectAnalysis(project_name,  1, root=results_path)
        
        commitsWithBuildSystem = 0
        
        for idx, commit_dict in enumerate(pa.csvDict.values()):
            build=pa.get_build_file(idx, commit_dict['commit'])
            if any(b['build_system'] in ['Maven', 'Gradle', 'Ant'] for b in build['builds_checked']):
                commitsWithBuildSystem += 1
            
            
        projects.append([
            pa.project,
            commitsWithBuildSystem,
            pa.df()[(pa.df()['build'] == 'SUCCESS')]['id'].count(),
            pa.df()[(pa.df()['build'] == 'FAIL')]['id'].count(),
            pa.df()[(pa.df()['build'] == 'NO')]['id'].count(),
            pa.df()['id'].count(),
            pa.df()[(pa.df()['build'] == 'SUCCESS')]['id'].count() * 100/ commitsWithBuildSystem
        ])

    df = pd.DataFrame(projects, columns = ['Project', 'Buildable', 'Success', 'Fail', 'No', 'TotalCommits', '% SUCCESS']).round(decimals=2)
    print("Projects: %d"%df['Project'].count())
    df = df.sort_values(by=['Project'])
    return df

In [10]:
df = get_projects_resume()
df.to_csv('GitHubStatusCheckerResults.csv', index=False)

Projects: 80
